# 第2章　分散表現

## 2.2 cos類似度

### cosine_simirarityモジュールによるcos類似度

In [2]:
# CountVectorizerモジュールの入力形式に変換する関数
import janome
from janome.tokenizer import Tokenizer

t = Tokenizer()

def make_corpus(documents):
    result_corpus = []
    
    for adocument in documents:
        words = [token for token in t.tokenize(adocument,wakati=True)]
        text = " ".join(words)
        result_corpus.append(text)
    return result_corpus

In [3]:
document1 = '私は秋田犬が大好きです。私の好きな犬は秋田犬です。'
document2 = '私は犬が苦手です。犬より猫のほうが好きです。'
document3 = '私は本が好きです。毎日本を読みます。'
document4 = '私は数学が苦手です。'
document5 = '私は運動が得意で、走るのが速いです。'
document6 = '私は毎日8時間寝ています。'
document7 = '私は肉と魚が好きです。嫌いな食べ物はピーマンです。'
document8 = '私は毎朝電車で通勤しています。満員電車は嫌いです。'
document9 = '今日はいい天気です。日差しがまぶしい。'
document10 = '明日は雨が降るかもしれません。'

documents = [document1,document2,document3,document4,document5,document6,document7,document8,document9,document10]
dict = make_corpus(documents) 
print(dict)


['私 は 秋田 犬 が 大好き です 。 私 の 好き な 犬 は 秋田 犬 です 。', '私 は 犬 が 苦手 です 。 犬 より 猫 の ほう が 好き です 。', '私 は 本 が 好き です 。 毎 日本 を 読み ます 。', '私 は 数学 が 苦手 です 。', '私 は 運動 が 得意 で 、 走る の が 速い です 。', '私 は 毎日 8 時間 寝 て い ます 。', '私 は 肉 と 魚 が 好き です 。 嫌い な 食べ物 は ピーマン です 。', '私 は 毎朝 電車 で 通勤 し て い ます 。 満員 電車 は 嫌い です 。', '今日 は いい 天気 です 。 日差し が まぶしい 。', '明日 は 雨 が 降る かも しれ ませ ん 。']


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b',sublinear_tf=True)
X = vectorizer.fit_transform(dict)
X.shape

(10, 55)

In [6]:
#次元数を減らしてみる
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=3,n_iter=5,random_state=42)
newX = svd.fit_transform(X)
print(newX)
print(newX.shape)
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())

[[ 0.6755768  -0.27843101 -0.26856757]
 [ 0.71449893 -0.3433819  -0.1981751 ]
 [ 0.46417642  0.19180123 -0.02346579]
 [ 0.60150519 -0.11702002  0.07424716]
 [ 0.50587724 -0.02412961  0.11104759]
 [ 0.26155699  0.7271639  -0.07386861]
 [ 0.53685042  0.01953388 -0.04135825]
 [ 0.37314351  0.67744546 -0.02297382]
 [ 0.30515288 -0.07094008  0.48275125]
 [ 0.16987888 -0.02828892  0.79982474]]
(10, 3)
[0.03800946 0.15283318 0.12135128]
0.3121939206181686


In [10]:
# cos類似度を求める
from sklearn.metrics.pairwise import cosine_similarity
A = newX[0]
B = newX[9]
print(A)
print(A.reshape(1,-1))
print(cosine_similarity(A.reshape(1,-1),B.reshape(1,-1)))

[ 0.6755768  -0.27843101 -0.26856757]
[[ 0.6755768  -0.27843101 -0.26856757]]
[[-0.1447005]]


In [12]:
# cosine_similarityの第2引数を省略
# cos類似度を求める
from sklearn.metrics.pairwise import cosine_similarity
result = cosine_similarity(newX)
print(result)
result.shape

[[ 1.          0.99277153  0.6808168   0.87194947  0.78945428 -0.00980307
   0.87817903  0.11560315  0.21484288 -0.1447005 ]
 [ 0.99277153  1.          0.65826394  0.90257702  0.82075836 -0.07594427
   0.87463065  0.06095802  0.31200454 -0.0410048 ]
 [ 0.6808168   0.65826394  1.          0.82169839  0.87300438  0.67282425
   0.93728093  0.78059133  0.40334546  0.1328746 ]
 [ 0.87194947  0.90257702  0.82169839  1.          0.9853611   0.13930184
   0.95484607  0.3003854   0.640971    0.32647966]
 [ 0.78945428  0.82075836  0.87300438  0.9853611   1.          0.26476365
   0.95403579  0.42341943  0.70274471  0.41357575]
 [-0.00980307 -0.07594427  0.67282425  0.13930184  0.26476365  1.
   0.37697738  0.98542669 -0.01663172 -0.0554525 ]
 [ 0.87817903  0.87463065  0.93728093  0.95484607  0.95403579  0.37697738
   1.          0.51453083  0.4594673   0.13059253]
 [ 0.11560315  0.06095802  0.78059133  0.3003854   0.42341943  0.98542669
   0.51453083  1.          0.12287832  0.04083389]
 [ 0.214

(10, 10)

### PyTorchによるcos類似度の実装

In [13]:
!pip3 install torch torchvision torchaudio

     -------------------------------------- 161.8/161.8 MB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 7.5 MB/s eta 0:00:00
     -------------------------------------- 969.6/969.6 kB 5.6 MB/s eta 0:00:00


In [14]:
import torch
print(torch.__version__)

1.12.1+cpu


In [18]:
import torch
import torch.nn.functional as F
A = torch.FloatTensor(newX[0])
B = torch.FloatTensor(newX[9])
F.cosine_similarity(A,B,dim=0)

tensor(-0.1447)

## 2.3 word2vec

### gensimモジュールによるword2vecの作成

In [22]:
# 分かち書きファイルの作成
import janome
from janome.tokenizer import Tokenizer

t = Tokenizer()

datafile = "melos.txt"
wakatifile = "wakati.txt"

with open(wakatifile,'w',encoding='UTF-8') as f2:
    with open(datafile,'r',encoding='UTF-8') as f1:
        for line in f1:
            for token in t.tokenize(line,wakati=True):
                f2.write(token + " ")
            f2.write('\n')

In [35]:
from gensim.models import word2vec
from gensim.models import Word2Vec
sentences = word2vec.LineSentence(wakatifile)
model = Word2Vec(sentences)
print(model['犬'])

TypeError: 'Word2Vec' object is not subscriptable